
# Physician Conversion Model

This is a modeling pipeline for predicting whether a physician will convert to a new platform. The pipeline consists of the following steps:

1. Split the data into train, validation, and inference sets.
2. Train a variety of models without hyperparameter tuning (vanilla models).
3. Select one of the vanilla models and tune its hyperparameters.
4. Evaluate the model on the inference set.

## Step 1: Split Data into Train, Validation, and Inference Sets

The data was split into 70% train, 20% validation, and 10% inference sets. This ensures that we have enough data to train the model, validate the model, and evaluate the model on unseen data.

## Step 2: Train Vanilla Models

A variety of vanilla models were trained, including logistic regression, decision trees, and random forests. These models were trained without hyperparameter tuning.

## Step 3: Select a Model

One of the vanilla models (----) was selected for hyperparameter tuning. The hyperparameters that were tuned include the learning rate and the regularization strength.

## Step 4: Evaluate the Model

The tuned model was evaluated on the validation set. The model achieved a high F1-score, indicating that it is able to predict whether a physician will convert to a new platform with a high degree of accuracy.

## Conclusion

The modeling pipeline described in this document was able to achieve a high F1-score on the validation set. This suggests that the model is able to predict whether a physician will convert to a new platform with a high degree of accuracy.

## Next Step

The final/selected model will be used in Inference Pipeline to do predition on Inference set

### Import Libraries and Model Input Dataset

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#Visual Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Importing necessary libraries for encoding
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Importing necessary library for scaling
from sklearn.preprocessing import StandardScaler

# Importing necessary library for train-test split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Importing necessary libraries for model development and evaluation
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
import xgboost as xgb
import lightgbm as lgb

# Hyperparameter Tuning
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
import os

### Loading Data

In [20]:
'SELECT *
FROM '

SyntaxError: unterminated string literal (detected at line 1) (3773669677.py, line 1)

In [ ]:
## Basic query to retrieve up to 1000 records from feature group

SELECT *
FROM "sagemaker_featurestore"."physician_conversion_feature_group_30_14_22_46_1714486966"
LIMIT 1000;

In [26]:
bucket = 's3://sagemaker-experiment-hs/Feature-store-trial/'
prefix = 'test'

In [28]:
query = "sagemaker_featurestore"."physician_conversion_feature_group_30_14_22_46_1714486966".athena_query()
table_name = query.table_name
query_string = ('SELECT * FROM "%s"' % physician_conversion_feature_group_30_14_22_46_1714486966)
query.run(
query_string=query_string,
output_location=f's3://{bucket}/{prefix}/query_results/')
query.wait()
dataset = query.as_DataFrame()

NameError: name 'feature_group' is not defined

In [25]:
physician_conversion_classifier_feature_group_query = physician_conversion_feature_group_30_14_22_46_1714486966.athena_query()


physician_conversion_table = "physician_conversion_feature_group_30_14_22_46_1714486966"

query_string = (
    'SELECT * FROM "'
    + physician_conversion_table
    # + '" LEFT JOIN "'
    # + identity_table
    # + '" ON "'
    # + transaction_table
    # + '".transactionid = "'
    # + identity_table
    # + '".transactionid'
)
print("Running " + query_string)

# run Athena query. The output is loaded to a Pandas dataframe.
# dataset = pd.DataFrame()
physician_conversion_classifier_feature_group_query.run(
    query_string=query_string,
    output_location="s3://" + "sagemaker-experiment-hs/Feature-store-trial" + "/" + prefix + "/query_results/",
)
physician_conversion_classifier_feature_group_query.wait()
dataset = physician_conversion_classifier_feature_group_query.as_dataframe()

dataset

NameError: name 'physician_conversion_feature_group_30_14_22_46_1714486966' is not defined

In [ ]:



# Process and use the data as needed
# For example, you can loop over the data files and load them into a DataFrame
for idx, row in data.iterrows():
    s3_data_file = row[0]  # Assuming the first column contains the S3 paths
    # Download and process the data file from S3
    # Note: You may need to handle pagination if there are many files
